In [ ]:
! wget https://raw.githubusercontent.com/facebookresearch/SentEval/master/data/probing/subj_number.txt
! wget https://raw.githubusercontent.com/facebookresearch/SentEval/master/data/probing/tree_depth.txt
! wget https://raw.githubusercontent.com/facebookresearch/SentEval/master/data/probing/sentence_length.txt

In [ ]:
from transformers import T5ForConditionalGeneration, T5EncoderModel, T5Config, T5Tokenizer
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

enc = LabelEncoder()
device = 'cuda:0'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #  модели лежат на диске для быстрого доступа

In [ ]:
depth = pd.read_csv('tree_depth.txt', sep='\t', header=None)
subjnum = pd.read_csv('subj_number.txt', sep='\t', header=None)
length = pd.read_csv('sentence_length.txt', sep='\t', header=None)

In [ ]:
def load_model(checkpoint):
    m = torch.load(f'/content/gdrive/My Drive/model_{checkpoint} (1).pth')
    model = T5ForConditionalGeneration(T5Config(output_hidden_states=True))
    model.load_state_dict(m['model_state_dict'])
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model.to(device)
    return model, tokenizer

In [ ]:
def get_emb(sent, model, tokenizer):
  with torch.no_grad():
    enc = tokenizer(sent, padding=True, truncation=True, return_tensors='pt')
    enc.to(device)
    output = model.encoder(
      input_ids=enc['input_ids'], 
      attention_mask=enc["attention_mask"], 
      return_dict=True
    )
    # get the final hidden states
    emb = output.hidden_states
    return [torch.mean(e, 1).squeeze(0).cpu().numpy() for e in emb]

In [ ]:
def classify(X_train, X_test, y_train, y_test):
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    return (y_pred, accuracy_score(y_test, y_pred), precision_score(y_test, y_pred, average='micro'),
    recall_score(y_test, y_pred, average='micro'), f1_score(y_test, y_pred, 'micro'))

In [ ]:
def probe(epoche, df, scores, pred):
    model, tokenizer = load_model(epoche)
    print('Model is loaded')
    TRAIN = df[df.iloc[:,0] == 'tr']
    TEST = df[df.iloc[:,0] == 'te']
    X_train = TRAIN.iloc[:,2].apply(lambda x: get_emb(x, model, tokenizer))
    X_test = TEST.iloc[:,2].apply(lambda x: get_emb(x, model, tokenizer))
    print('Embeddings are calculated')
    enc.fit(df.iloc[:,1])
    y_train = enc.transform(TRAIN.iloc[:,1])
    y_test = enc.transform(TEST.iloc[:,1])
    for a in tqdm(range(7)):
        train = np.array([x[a].tolist() for x in X_train.to_list()])
        test = np.array([x[a].tolist() for x in X_test.to_list()])
        sc = classify(train, test, y_train, y_test)
        pred.append(sc[0])
        scores.append(sc[1:])
    print('Score is calculated')
    return scores, pred

In [ ]:
pred = []
scores = []
# checkpoint = number of checkpoint
scores, pred = probe(checkpoint, length, scores, pred)

In [ ]:
with open('pred.txt', 'a') as f:
      for i in pred:
        f.write(','.join(list(map(str, i))))
    f.write('\n')

In [ ]:
scores = [[i,] + list(a) for i, a in enumerate(scores)]
sc = pd.DataFrame(scores, columns=['layer', 'accuracy', 'precision', 'recall', 'f1-score'])
sc.to_csv('new_scores.csv', mode='a', header=False)